# Concatenating Tables with Set-Like Operations

One of the two way of combining two tables is to stack one table on top of the other.  When stacking two tables on top of one another, we need to decide

1. If we combine columns based on position or name (and if combining by name, what do we do with mismatches?)
2. How to decide which rows to keep.  In this case, we will take some guidance from SQL clauses.

## Three Types of Operations

* **Union:** Keeps rows from either table.
* **Intersection:** Only keeps common columns
* **Set Difference/Except:** Keep rows from the left table *except* those in the right table.

## Set Operations in Action 

<img src="./img/table_verbs_set.gif" width=800>

## All Operations Match by Position

All operations

* Match columns by position
* Require same number/type of columns

## Distinct Versus All

* **UNION/INTERSECT/SET DIFFERENE** are **DISTINCT**
    * Only keeps distinct rows, removing duplicates.
* **UNION ALL/INTERSECT ALL/SET DIFFERENCE ALL**
    * Keeps duplicate rows

In [1]:
import pandas as pd
from dfply import *

In [2]:
sales_may = pd.read_csv('./data/auto_sales_may.csv')
sales_may

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9


In [3]:
sales_apr = pd.read_csv('./data/auto_sales_apr.csv')
sales_apr

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


## Unions with `dfply`

Use `left_table >> union(right_table)`

In [4]:
sales_may >> union(sales_apr)

/home/nathan/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/dfply/set_ops.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stacked = df.append(other)


,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


## `dfply.union` is distinct

Since Ann have the same sales each month, her row only included one row.  Note that we can use `keep='last'` to `keep='first'` to determine which row is kept.

In [5]:
sales_may >> union(sales_apr, keep='last')

/home/nathan/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/dfply/set_ops.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stacked = df.append(other)


,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


In [6]:
sales_may >> union(sales_apr, keep='first')

/home/nathan/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/dfply/set_ops.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stacked = df.append(other)


,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


## Making `union_all`

We can use `pd.concat` to perform a `UNION ALL`

In [7]:
from more_dfply import union_all
sales_may >> union_all(sales_apr)

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9
4,0,Ann,22,18,15,12
5,1,Bob,19,12,17,20
6,2,Yolanda,19,8,32,15
7,3,Xerxes,12,23,18,9


## Adding a month column

Another way to keep both of Ann's sales rows is adding a month column (which we should probably do anyway).

In [8]:
sales_may >> mutate(month = 'May') >> union(sales_apr >> mutate(month = 'April'))

/home/nathan/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/dfply/set_ops.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stacked = df.append(other)


,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck,month
0,0,Ann,22,18,15,12,May
1,1,Bob,20,14,6,24,May
2,2,Yolanda,19,10,28,17,May
3,3,Xerxes,11,27,17,9,May
0,0,Ann,22,18,15,12,April
1,1,Bob,19,12,17,20,April
2,2,Yolanda,19,8,32,15,April
3,3,Xerxes,12,23,18,9,April


## Finding common rows with `dfply.intersect`

In [9]:
sales_may >> intersect(sales_apr)

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12


## Finding rows unique to the left table.

Use `left_table >> dfply.set_diff(right_table)`

In [10]:
sales_may >> set_diff(sales_apr)

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9


# Working with many and/or large files

In this section, we will take a look at techniques for working with many files, as well as large files.

In [11]:
import pandas as pd
from dfply import *

## Baseball data

We will be using the [Baseball Databank](https://github.com/chadwickbureau/baseballdatabank), make sure you have these data cloned into `./data/baseball`.

In [12]:
!git clone https://github.com/chadwickbureau/baseballdatabank.git ./data/baseball

fatal: destination path './data/baseball' already exists and is not an empty directory.


## Working with many files.

* Use `glob.glob` to find all files that match a pattern
* Convert all files to `pd.DataFrames`
* Store the `df` in a list or dictionary

## What the heck is a `glob`

`glob.glob`

* Takes a path regular expression
* Returns a list of files that match the patterm
* Relative paths!

## Store in `dict` or `list`?

* Natural sequence/order? $\rightarrow$ `list`
    *  Example: Lakes data and years are a natural sequence
* Easier to refer by name? $\rightarrow$ `dict`
    * Baseball files have no order and easier to refer to by name

## Example 1 - Using `glob` to read and combine the sales data

Using `glob` with a `list` to automate reading an combining files 

#### Step 1 - Get the file names

In [12]:
from glob import glob
sales_files = glob('./data/auto_sales_*.csv')
sales_files

['./data/auto_sales_apr.csv', './data/auto_sales_may.csv']

#### Step 2 - Read the files into a list of data frames

In [14]:
sales_by_month = [pd.read_csv(f) for f in sales_files]

 #### Inspect each data from with head

In [15]:
[df.head(2) for df in sales_by_month]

[   Unnamed: 0 Salesperson  Compact  Sedan  SUV  Truck
 0           0         Ann       22     18   15     12
 1           1         Bob       19     12   17     20,
    Unnamed: 0 Salesperson  Compact  Sedan  SUV  Truck
 0           0         Ann       22     18   15     12
 1           1         Bob       20     14    6     24]

#### Step 3 - Pull off the month from the file names and repackage as a `dict`

In [16]:
import re

MONTH_RE = re.compile(r'^\./data/auto_sales_([a-zA-Z_]*)\.csv$')
get_month = lambda p: MONTH_RE.match(p).group(1) 
month_names = lambda files: [get_month(p) for p in files]
month_names(sales_files)

['apr', 'may']

In [17]:
month_name_and_file = list(zip(month_names(sales_files), sales_files))
month_name_and_file

[('apr', './data/auto_sales_apr.csv'), ('may', './data/auto_sales_may.csv')]

#### Now repackage with a `list` comprehension

Note that we will need the month name later, so we are storing it in a `tuple` with the data frame for now.

In [18]:
sales_by_month = [(mon,pd.read_csv(file)) for mon, file in month_name_and_file]

In [19]:
[(mon, df.head(2)) for mon, df in sales_by_month]

[('apr',
     Unnamed: 0 Salesperson  Compact  Sedan  SUV  Truck
  0           0         Ann       22     18   15     12
  1           1         Bob       19     12   17     20),
 ('may',
     Unnamed: 0 Salesperson  Compact  Sedan  SUV  Truck
  0           0         Ann       22     18   15     12
  1           1         Bob       20     14    6     24)]

#### Step 4 - Add a month column to each file

Notice that we need to put the `dfply` pipe *inside* the `list` comprehension to allow access to the names.

In [20]:
sale_files_with_month = [(df
                          >> mutate(month = mon)
                         )
                         for mon, df in sales_by_month
                        ]

In [21]:
[df.head(2) for df in sale_files_with_month]

[   Unnamed: 0 Salesperson  Compact  Sedan  SUV  Truck month
 0           0         Ann       22     18   15     12   apr
 1           1         Bob       19     12   17     20   apr,
    Unnamed: 0 Salesperson  Compact  Sedan  SUV  Truck month
 0           0         Ann       22     18   15     12   may
 1           1         Bob       20     14    6     24   may]

#### Step 5 - Combine the files using `pd.concat`

Note that `pd.concat` is `dfply.union_all`

In [58]:
?pd.concat

In [57]:
combined_files = pd.concat(sale_files_with_month)
combined_files

,Unnamed: 0,Salesperson,Compact,Sedan,SUV,Truck,month
0,0,Ann,22,18,15,12,apr
1,1,Bob,19,12,17,20,apr
2,2,Yolanda,19,8,32,15,apr
3,3,Xerxes,12,23,18,9,apr
0,0,Ann,22,18,15,12,may
1,1,Bob,20,14,6,24,may
2,2,Yolanda,19,10,28,17,may
3,3,Xerxes,11,27,17,9,may


## <font color="red"> Exercise 2.10.1</font>

In the data folder, you will find 6 files that contain a sample 100,000 rows from the uber data for the month apr14-sep14.  Perform the following tasks:

1. Use `glob` to get all 6 file paths.
2. Use a regular expression to create a `lambda` function that pulls the month from the files.
3. Read the 6 data frames into a `list` of `tuples` containing the month name corresponding data frame.
4. Add the month column each data frame using a pipe inside of a comprehension.
5. Use `pd.concat` to combine these 6 data frames into one combined `df`

In [ ]:
uber_files = glob('./data/uber-raw-data-[A-Za-z0-9]*-sample.csv')
uber_files

In [ ]:
monthly_uber = [pd.read_csv(f) for f in uber_files]
[df.head(2) for df in monthly_uber]

In [ ]:
MONTH_RE = re.compile(r'^\./data/uber-raw-data-([a-zA-Z0-9]*)-sample\.csv$')
get_month = lambda p: MONTH_RE.match(p).group(1) 
month_names = lambda files: [get_month(p) for p in files]
month_names(uber_files)

In [ ]:
month_name_and_file = list(zip(month_names(uber_files), uber_files))
month_name_and_file

In [ ]:
uber_by_month = [(mon,pd.read_csv(file)) for mon, file in month_name_and_file]
[(mon, df.head(2)) for mon, df in uber_by_month]

In [42]:
uber_files_with_month = [(df
                          >> mutate(month = mon)
                         )
                         for mon, df in uber_by_month
                        ]

In [45]:
combined_files = pd.concat(uber_files_with_month)
combined_files

,Date/Time,Lat,Lon,Base,month
0,4/18/2014 21:38:00,40.7359,-73.9852,B02682,apr14
1,4/23/2014 15:19:00,40.7642,-73.9543,B02598,apr14
2,4/10/2014 7:15:00,40.7138,-74.0103,B02598,apr14
3,4/11/2014 15:23:00,40.7847,-73.9698,B02682,apr14
4,4/7/2014 17:26:00,40.6460,-73.7767,B02598,apr14
...,...,...,...,...,...
99995,9/13/2014 16:51:00,40.7959,-73.9326,B02617,sep14
99996,9/22/2014 22:48:00,40.7541,-73.9490,B02617,sep14
99997,9/11/2014 22:38:00,40.7292,-73.9990,B02512,sep14
99998,9/10/2014 10:36:00,40.7706,-73.8671,B02598,sep14


## Example 2 - Reading and joining the baseball database using `dict`

**Task:** Collect the number of total hits for each batters in the 2010 season join on their first and last name.

In the second example, we will store the data frames in a `dict`, which will make it easier to join the files by ne

#### Step 1 - Get the files names

In [46]:
from glob import glob
files = glob('./data/baseball/core/*.csv')
files

['./data/baseball/core/AllstarFull.csv',
 './data/baseball/core/Appearances.csv',
 './data/baseball/core/AwardsManagers.csv',
 './data/baseball/core/AwardsPlayers.csv',
 './data/baseball/core/AwardsShareManagers.csv',
 './data/baseball/core/AwardsSharePlayers.csv',
 './data/baseball/core/Batting.csv',
 './data/baseball/core/BattingPost.csv',
 './data/baseball/core/CollegePlaying.csv',
 './data/baseball/core/Fielding.csv',
 './data/baseball/core/FieldingOF.csv',
 './data/baseball/core/FieldingOFsplit.csv',
 './data/baseball/core/FieldingPost.csv',
 './data/baseball/core/HallOfFame.csv',
 './data/baseball/core/HomeGames.csv',
 './data/baseball/core/Managers.csv',
 './data/baseball/core/ManagersHalf.csv',
 './data/baseball/core/Parks.csv',
 './data/baseball/core/People.csv',
 './data/baseball/core/Pitching.csv',
 './data/baseball/core/PitchingPost.csv',
 './data/baseball/core/Salaries.csv',
 './data/baseball/core/Schools.csv',
 './data/baseball/core/SeriesPost.csv',
 './data/baseball/core

* Only need the `Batting.csv` and `People.csv`.  
* Narrow with a RegEx

In [47]:
import re
needed_file = re.compile(r'./data/baseball/core/(Batting|People).csv')

needed_files = [f for f in files if needed_file.match(f)]
needed_files

['./data/baseball/core/Batting.csv', './data/baseball/core/People.csv']

#### Step 2 - Make helper functions to get the name from path

In [48]:
import re
FILE_NAME_RE = re.compile(r'^\./data/baseball/core/([a-zA-Z_]*)\.csv$')
file_name = lambda p: FILE_NAME_RE.match(p).group(1) 
file_names = lambda files: [file_name(p) for p in files]
file_names(needed_files)

['Batting', 'People']

#### Step 3 - Use a comprehension to read in all files

**Note:** The data is small (< 10mb total) so it is safe to read all at once.

In [ ]:
dfs = {name:pd.read_csv(path) for name, path in zip(file_names(needed_files), needed_files)}
dfs['Batting'].head()

In [ ]:
dfs['People'].head()

#### Step 4 - Preprocess each file.

In [ ]:
# Filter, select, and aggregate hits for 2010.
hits_in_2010_raw = (dfs['Batting']
                   >> select(X.yearID, X.playerID, X.H)
                   >> filter_by(X.yearID == 2010)
                   >> group_by(X.playerID)
                   >> summarise(total_hits = mean(X.H))
                   )
hits_in_2010_raw.head(2)

In [ ]:
# Grab the first and last names from People.

player_names = (dfs['People']
         >> select(X.playerID, X.nameFirst, X.nameLast))
player_names.head(2)

#### Step 4 -- Join the tables

In [ ]:
hits_in_2010 = (hits_in_2010_raw 
                >> left_join(player_names, by='playerID')
                >> drop(X.playerID)
               )
hits_in_2010.head()

## <font color="red"> Exercise 2.10.2 </font>

We want to get the total hits allowed for all pitchers during the 2000-2010 seasons.  Use `glob` and a `dict` to collect this information into a table that includes the players first and last names.

In [63]:
needed_file = re.compile(r'./data/baseball/core/(Pitching|People).csv')

needed_files = [f for f in files if needed_file.match(f)]
needed_files

['./data/baseball/core/People.csv', './data/baseball/core/Pitching.csv']

In [64]:
FILE_NAME_RE = re.compile(r'^\./data/baseball/core/([a-zA-Z_]*)\.csv$')
file_name = lambda p: FILE_NAME_RE.match(p).group(1) 
file_names = lambda files: [file_name(p) for p in files]
file_names(needed_files)

['People', 'Pitching']

In [65]:
dfs = {name:pd.read_csv(path) for name, path in zip(file_names(needed_files), needed_files)}

In [ ]:
dfs['Pitching'].head()

In [ ]:
dfs['People'].head()

In [ ]:
player_names = (dfs['People']
         >> select(X.playerID, X.nameFirst, X.nameLast))
player_names

pandas.core.series.Series

In [ ]:
total_hits_allowed_2000_to_2010 = (dfs['Pitching']
                   >> select(X.yearID, X.playerID, X.H)
                   >> filter_by(X.yearID <= 2010)
                   >> filter_by(X.yearID >= 2000)
                   >> group_by(X.playerID)
                   >> summarize(total_hits_allowed = X.H.sum())
                   )
total_hits_allowed_2000_to_2010

In [96]:
total_hits_allowed_2000_to_2010_with_names = (total_hits_allowed_2000_to_2010
                                                >> left_join(player_names))
total_hits_allowed_2000_to_2010_with_names

,playerID,total_hits_allowed,nameFirst,nameLast
0,aardsda01,231,David,Aardsma
1,abadfe01,14,Fernando,Abad
2,abbotpa01,502,Paul,Abbott
3,abreuwi01,57,Winston,Abreu
4,accarje01,203,Jeremy,Accardo
...,...,...,...,...
1866,zimmeje02,128,Jeff,Zimmerman
1867,zimmejo02,126,Jordan,Zimmermann
1868,zinkch01,11,Charlie,Zink
1869,zitoba01,1959,Barry,Zito
